<a href="https://colab.research.google.com/github/drewamorbordelon/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/LS_DS19_RidgeRegression_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [ ]:
%%capture
import sys
import pandas as pd

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:

import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')
               

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23040 entries, 0 to 23039
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  int64  
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNITS                

In [ ]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [ ]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [ ]:
# Set 'SALE_DATE' to pd.to_datetime  format
print(df.shape)
df.head()
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
df = df.set_index('SALE_DATE')
df.head()

(23040, 21)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0
2019-01-01,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0
2019-01-01,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0
2019-01-01,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0
2019-01-01,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0


In [ ]:
df.ZIP_CODE.value_counts()

11235.0    336
11234.0    335
10314.0    331
11375.0    327
11215.0    298
          ... 
10006.0     12
10039.0     12
10282.0      6
10105.0      1
11227.0      1
Name: ZIP_CODE, Length: 184, dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 23040 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         23040 non-null  object 
 1   NEIGHBORHOOD                    23040 non-null  object 
 2   BUILDING_CLASS_CATEGORY         23040 non-null  object 
 3   TAX_CLASS_AT_PRESENT            23039 non-null  object 
 4   BLOCK                           23040 non-null  int64  
 5   LOT                             23040 non-null  int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       23039 non-null  object 
 8   ADDRESS                         23040 non-null  object 
 9   APARTMENT_NUMBER                5201 non-null   object 
 10  ZIP_CODE                        23039 non-null  float64
 11  RESIDENTIAL_UNITS               23039 non-null  float64
 12  COMMERCIAL_UNIT

In [ ]:
df = df.loc[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'] 
# df = df.drop(['EASE-MENT','APARTMENT_NUMBER', 'ADDRESS', 'ZIP_CODE', 
              # 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 
              # 'BUILDING_CLASS_AT_PRESENT', 'TAX_CLASS_AT_PRESENT', 
              # 'BUILDING_CLASS_AT_TIME_OF_SALE'], axis=1)
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0
2019-01-01,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0
2019-01-01,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000


In [ ]:
df = df.loc[df['SALE_PRICE'] > 100000]
df = df.loc[df['SALE_PRICE'] < 2000000]

df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].str.replace(',','')
df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(float)

print(df.shape)
df.head()


(3151, 20)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE_DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1,A5,620000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BLOCK                           3151 non-null   int64  
 5   LOT                             3151 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 8   ADDRESS                         3151 non-null   object 
 9   APARTMENT_NUMBER                1 non-null      object 
 10  ZIP_CODE                        3151 non-null   float64
 11  RESIDENTIAL_UNITS               3151 non-null   float64
 12  COMMERCIAL_UNITS

In [ ]:
# Set target and separate it from the feature matrix
target = 'SALE_PRICE'
y = df[target]
X = df.drop([target]+['EASE-MENT','ZIP_CODE', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
                    'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'LOT',
                    'ADDRESS', 'APARTMENT_NUMBER', 'BLOCK',
                    'BUILDING_CLASS_AT_TIME_OF_SALE', 'TAX_CLASS_AT_TIME_OF_SALE'], axis=1) 
#              

# OneHotEncoder(cols=['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
#                     'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT',
#                     'ADDRESS', 'APARTMENT_NUMBER',
#                     'BUILDING_CLASS_AT_TIME_OF_SALE'],

print(y.shape)
print(X.shape)

(3151,)
(3151, 7)


In [ ]:
# Split dataset into training data and validation data
cutoff= '2019-04-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_val, y_val = X.loc[~mask], y.loc[~mask]

In [ ]:
# Check work by making sure the shapes/ dimensions are equal
assert X_train.shape[0] + X_val.shape[0] == X.shape[0]

##Establish a Baseline

In [ ]:
y_train.mean()

621573.7423214999

In [ ]:
from sklearn.metrics import mean_absolute_error

print("Baseline MAE:", mean_absolute_error(y_train, [y_train.mean()]*len(y_train)))

Baseline MAE: 214721.52773001452


##Build a Model

Our model will have three components:

1. `transformer_1`: `OneHotEncoder`
2. `transformer_2`: `SelectKBest`
3. `predictor`: `LinearRegression`

In [ ]:
# import OneHotEncoder and LinearRegression
from sklearn.linear_model import LinearRegression
from category_encoders import OneHotEncoder

In order to build our model, we need to *transform* our data.

# 1. `OneHotEncoder`

In [ ]:
X_train.columns

Index(['BOROUGH', 'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

###**Instantiate** Transformer

In [ ]:
transformer_1 = OneHotEncoder(use_cat_names=True)

###**Fit** Transformer to the Data

In [ ]:
transformer_1.fit(X_train)

OneHotEncoder(cols=['BOROUGH'], drop_invariant=False, handle_missing='value',
              handle_unknown='value', return_df=True, use_cat_names=True,
              verbose=0)

###**Transform** our **Training Data**

In [ ]:
XT_train = transformer_1.transform(X_train)

In [ ]:
# Check XT_train.columns
XT_train.columns

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'RESIDENTIAL_UNITS', 'COMMERCIAL_UNITS', 'TOTAL_UNITS',
       'LAND_SQUARE_FEET', 'GROSS_SQUARE_FEET', 'YEAR_BUILT'],
      dtype='object')

In [ ]:
X_train.head()

,BOROUGH,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT
SALE_DATE,,,,,,,
2019-01-01,3,1.0,0.0,1.0,6800.0,1325.0,1930.0
2019-01-01,4,1.0,0.0,1.0,4000.0,2001.0,1940.0
2019-01-02,2,1.0,0.0,1.0,3500.0,2043.0,1925.0
2019-01-02,3,1.0,0.0,1.0,4000.0,2680.0,1899.0
2019-01-02,3,1.0,0.0,1.0,1710.0,1872.0,1940.0


In [ ]:
X_train['BOROUGH'].head()

SALE_DATE
2019-01-01    3
2019-01-01    4
2019-01-02    2
2019-01-02    3
2019-01-02    3
Name: BOROUGH, dtype: object

In [ ]:
XT_train[['BOROUGH_1',
         'BOROUGH_2',
         'BOROUGH_3',
         'BOROUGH_4',
         'BOROUGH_5']].head()

,BOROUGH_1,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5
SALE_DATE,,,,,
2019-01-01,0,0,1,0,0
2019-01-01,0,0,0,1,0
2019-01-02,0,1,0,0,0
2019-01-02,0,0,1,0,0
2019-01-02,0,0,1,0,0


## 2. `SelectKBest`

In [ ]:
# import SelectKBest
from sklearn.feature_selection import SelectKBest

###**Instantiate** Transformer

In [ ]:
transformer_2 = SelectKBest(k=3) # k = HYPERPARAMETER

###**Fit** Transformer to the Data

In [ ]:
transformer_2.fit(XT_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


SelectKBest(k=3, score_func=<function f_classif at 0x7fd0e15f6400>)

###**Transform** the **Training Data**

In [ ]:
XTT_train = transformer_2.transform(XT_train)

## 3. `LinearRegression`

###**Instantiate** Model

In [ ]:
predictor = LinearRegression()

###**Fit** Model to **Training Data**

In [ ]:
predictor.fit(XTT_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Check Metrics

In [ ]:
 # Make predictions (TRANSFORMED TRAINING DATA)
 y_pred = predictor.predict(XTT_train)

 # Calculate MAE
 print("training MAE:", mean_absolute_error(y_train, y_pred))

training MAE: 191600.98677115922


Validation Data Metric

In [ ]:
# FIRST have to transform validation data 
# DO NOT refit transformer with the validation data
XT_val = transformer_1.transform(X_val)
XTT_val = transformer_2.transform(XT_val)

# Make predictions (TRANSFORMED VALIDATION DATA)
y_pred = predictor.predict(XTT_val)

# Calculate Validation MAE
print("Validation MAE:", mean_absolute_error(y_val, y_pred))

Validation MAE: 183533.10620492848


# Ridge Regression

In [ ]:
# import Ridge
from sklearn.linear_model import Ridge

# Instantiate
predictor_2 = Ridge(alpha=.01, normalize=True)  # Alpha is a HYPERPARAMETER

# Fit to Training Data
predictor_2.fit(XTT_train, y_train)

Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)

In [ ]:
# Make Predictions (TRANSFORMED TRAINING DATA)
y_pred = predictor_2.predict(XTT_train)

# Calculate Training MAE
print("Training MAE:", mean_absolute_error(y_train, y_pred))

Training MAE: 191593.12466233733


In [ ]:
# Transform Validation/Test 
XT_val = transformer_1.transform(X_val)
XTT_val = transformer_2.transform(XT_val)

# Make Predictions (TRANSFORMED VALIDATION/TEST DATA)
y_pred = predictor_2.predict(XTT_val)

# Calculate Test/Validation MAE
print("Test MAE:", mean_absolute_error(y_val, y_pred))

Test MAE: 183591.29673441235


## R2 Score

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   BOROUGH                         3151 non-null   object 
 1   NEIGHBORHOOD                    3151 non-null   object 
 2   BUILDING_CLASS_CATEGORY         3151 non-null   object 
 3   TAX_CLASS_AT_PRESENT            3151 non-null   object 
 4   BLOCK                           3151 non-null   int64  
 5   LOT                             3151 non-null   int64  
 6   EASE-MENT                       0 non-null      float64
 7   BUILDING_CLASS_AT_PRESENT       3151 non-null   object 
 8   ADDRESS                         3151 non-null   object 
 9   APARTMENT_NUMBER                1 non-null      object 
 10  ZIP_CODE                        3151 non-null   float64
 11  RESIDENTIAL_UNITS               3151 non-null   float64
 12  COMMERCIAL_UNITS

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2507 entries, 2019-01-01 to 2019-03-30
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BOROUGH            2507 non-null   object 
 1   RESIDENTIAL_UNITS  2507 non-null   float64
 2   COMMERCIAL_UNITS   2507 non-null   float64
 3   TOTAL_UNITS        2507 non-null   float64
 4   LAND_SQUARE_FEET   2507 non-null   float64
 5   GROSS_SQUARE_FEET  2507 non-null   float64
 6   YEAR_BUILT         2507 non-null   float64
dtypes: float64(6), object(1)
memory usage: 156.7+ KB


In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3151 entries, 2019-01-01 to 2019-04-30
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BOROUGH            3151 non-null   object 
 1   RESIDENTIAL_UNITS  3151 non-null   float64
 2   COMMERCIAL_UNITS   3151 non-null   float64
 3   TOTAL_UNITS        3151 non-null   float64
 4   LAND_SQUARE_FEET   3151 non-null   float64
 5   GROSS_SQUARE_FEET  3151 non-null   float64
 6   YEAR_BUILT         3151 non-null   float64
dtypes: float64(6), object(1)
memory usage: 196.9+ KB


In [ ]:
# Multiple Linear Regression
from sklearn.metrics import r2_score
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
print("Training R2:", model.score(X_train, y_train))
print("Validation R2:", model.score(X_val, y_val))

Training R2: 0.2371042810318732
Validation R2: 0.2512048325447708
